# Load libraries

In [ ]:
library(tidyverse)
library(ggsignif)

# Set up paths

In [ ]:
code_dir <- getwd()
project_dir <- file.path(code_dir, "..")
input_dir <- file.path(code_dir, "../sourcedata")
output_dir <- file.path(code_dir, "../output/descriptives_supratentorial")
plot_dir <- file.path(code_dir, "../output/plots_supratentorial")

# Check if the directories exists

for (dir in c(output_dir, plot_dir)) {
  # Check if the directory exists
  if (!dir.exists(dir)) {
    # Create the directory if it does not exist
    dir.create(dir)
    cat("Created directory:", dir, "\n")
  } else {
    cat("Directory already exists:", dir, "\n")
  }
}

# Load dataframes

In [ ]:
df_joined <- read_csv(file.path(input_dir, "df_joined.csv"))
df_esoc_norm <- read_csv(file.path(input_dir, "df_normalized.csv"))

# Figures

## Lesion volumes

### Load data

In [ ]:
df_patients_supratentorial_lesionvolumes <- df_joined[df_joined$group == "pat" & df_joined$lesion_location_category %in% c("cortical","subcortical","cortical_subcortical"), ]

# Remove subjects with missing DWI data
df_patients_supratentorial_lesionvolumes <- drop_na(df_patients_supratentorial_lesionvolumes, "lesionanalysis_lesion_mean_FW")

df_patients_supratentorial_lesionvolumes <- df_patients_supratentorial_lesionvolumes[, c("sub_id", "ses_id", "lesion_volume")]
df_patients_supratentorial_lesionvolumes$ses_id <- factor(df_patients_supratentorial_lesionvolumes$ses_id, levels = 1:4, labels = c("3-5 days", "1 month", "3 months", "1 year"))

### Plot lesion volume per time point

In [ ]:
# Create figure
df_patients_supratentorial_lesionvolumes %>% 
ggplot(aes(x=ses_id, y=lesion_volume, fill=ses_id)) + 
geom_boxplot(position = position_dodge(width=0.8), width=0.6) +
scale_fill_brewer(palette = "Greens") +
guides(fill = FALSE) +
theme_test() + labs(y = "Lesion volume [ml]", x = "Time point") -> lesion_volume_per_tp

# Display figure
lesion_volume_per_tp

# Save figure
ggsave(file.path(plot_dir, "lesion_volume_per_tp.png"), plot = lesion_volume_per_tp, dpi = 300, width = 6, height = 5)

### Calculate mean and SD of lesion volume per time point

In [ ]:
df_patients_supratentorial_lesionvolumes %>%
group_by(ses_id) %>%
summarise(mean_lesionvolume = mean(lesion_volume), sd_lesionvolume = sd(lesion_volume), observations = nrow(lesion_volume)) -> lesion_volumes

lesion_volumes

write.csv(lesion_volumes, file.path(output_dir, "df_lesion_volumes.csv"), row.names = FALSE)

## Boxplots of FW and FAt averaged across lesion and shells

### Load data

In [ ]:
df_average_ipsicontra_supratentorial <- read_csv(file.path(input_dir, "df_average_ipsicontra_supratentorial_data.csv"))
df_average_ipsicontra_supratentorial$ses_id <- factor(df_average_ipsicontra_supratentorial$ses_id, levels = 1:4, labels = c("3-5 days", "1 month", "3 months", "1 year"))

### Average FW ipilateral vs contralateral at different time point

In [ ]:
# Create figure
df_average_ipsicontra_supratentorial %>% 
ggplot(aes(x=ses_id, y=fw, fill=location_fw)) + 
geom_boxplot(position = position_dodge(width=0.8), width=0.6) +
scale_x_discrete(expand = expansion(add = 0.6)) +
scale_fill_brewer(palette="Blues", direction=-1) +
theme_test() + labs(y = "FW", x = "", fill = "Location") -> fw_ipsicontra

# Display figure
fw_ipsicontra

# Save figure
ggsave(file.path(plot_dir, "fw_ipsicontra_all_supratentorial_barplot.png"), plot = fw_ipsicontra, dpi = 300, width = 6, height = 5)

### Average FAt ipsilateral vs contralateral at different time point

In [ ]:
# Create figure
df_average_ipsicontra_supratentorial %>% 
ggplot(aes(x=ses_id, y=fat, fill=location_fat)) + 
geom_boxplot(position = position_dodge(width=0.8), width=0.6) +
scale_x_discrete(expand = expansion(add = 0.6)) +
scale_fill_brewer(palette="Reds", direction=-1) +
theme_test() + labs(y = expression(FA[T]), x = "", fill = "Location") -> fat_ipsicontra

# Display figure
fat_ipsicontra

# Save figure
ggsave(file.path(plot_dir, "fat_ipsicontra_all_supratentorial_barplot.png"), plot = fat_ipsicontra, dpi = 300, width = 6, height = 5)

## Relative diffusion measures averaged across regions of interest for each time point

### Load data

In [ ]:
df_patients_supratentorial_all <- df_esoc_norm[df_esoc_norm$group == "pat" & df_esoc_norm$lesion_location_category %in% c("cortical","subcortical","cortical_subcortical"), ]
df_patients_supratentorial_all <- drop_na(df_patients_supratentorial_all, "lesionanalysis_lesion_mean_FW")
df_patients_supratentorial_all$ses_id <- factor(df_patients_supratentorial_all$ses_id, levels = 1:4, labels = c("3-5 days", "1 month", "3 months", "1 year"))
df_patients_supratentorial_all$location <- factor(df_patients_supratentorial_all$location, levels = c("lesion","2mm","4mm","6mm","8mm","10mm","12mm","14mm","16mm"))

### Calculate mean and SD of relative free-water

In [ ]:
df_fw_per_timepoint <- df_patients_supratentorial_all %>% group_by(ses_id) %>% summarise(mean(fw), sd(fw))

df_fw_per_timepoint

write.csv(df_fw_per_timepoint, file.path(output_dir, "df_fw_per_timepoint_supratentorial.csv"), row.names = FALSE)

### Plot average of relative free-water by time point

In [ ]:
# Create figure
df_fw_per_timepoint %>% 
ggplot(aes(x=ses_id, y=`mean(fw)`, fill = ses_id)) + 
geom_bar(color="black",position = "dodge", stat="summary", fun = "mean") +
geom_errorbar(aes(ymin=`mean(fw)`-`sd(fw)`, ymax=`mean(fw)`+`sd(fw)`), width=.2, position=position_dodge(.9)) +
geom_signif(comparisons = list(c("3-5 days", "3 months")), annotation = "***") +
geom_signif(comparisons = list(c("3-5 days", "1 year")), y_position=1.7,annotation = "***") +
geom_signif(comparisons = list(c("1 month", "1 year")), y_position=2.2, annotation = "***") +
geom_signif(comparisons = list(c("1 month", "3 months")), y_position=2,annotation = "***") +
geom_signif(comparisons = list(c("3 months", "1 year")), y_position=2.5,annotation = "***") +
geom_hline(yintercept=0, linetype = "solid", color = "red") +
scale_fill_brewer(palette="Blues", direction=-1) +
theme_test() + labs(y = "FW [(ipsi-contra)/contra]", x = "", fill = "Time Point") + 
ylim(-0.5,2.7) -> fw_normalized_barplot

# Display figure
fw_normalized_barplot

# Save figure
ggsave(file.path(plot_dir, "fw_normalized_all_supratentorial_barplot.png"), plot = fw_normalized_barplot, dpi = 300, width = 6, height = 5)

### Plot average of relative free-water by time point and shell

In [ ]:
# Create figure
df_patients_supratentorial_all %>% 
ggplot(aes(x=ses_id, y=fw, fill=location)) + 
annotate(geom = "rect", xmin= 0.55, xmax= 1.45, ymin=0, ymax = 0.109, color = "black", alpha=0.1) +
annotate(geom = "rect", xmin= 1.55, xmax= 2.45, ymin=0, ymax = 0.197, color = "black", alpha=0.2) +
annotate(geom = "rect", xmin= 2.55, xmax= 3.45, ymin=0, ymax = 0.389, color = "black", alpha=0.3) +
annotate(geom = "rect", xmin= 3.55, xmax= 4.45, ymin=0, ymax = 0.614, color = "black", alpha=0.4) +

geom_hline(yintercept=0, linetype = "solid", color = "red") +

geom_boxplot(position = position_dodge(width=0.8), width=0.6) +

annotate(geom = "text", x=0.65, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=0.74, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=0.83, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=0.92, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=1.01, y=-0.8, label="*",fontface="bold",color = "red") +
#annotate(geom = "text", x=1.28, y=-0.6, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=1.37, y=-0.8, label="*",fontface="bold",color = "red") +

annotate(geom = "text", x=1.65, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=1.74, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=1.92, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=2.01, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=2.19, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=2.37, y=-0.8, label="*",fontface="bold",color = "red") +

annotate(geom = "text", x=2.65, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=2.74, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=2.83, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=2.92, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.01, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.10, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.19, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.28, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.37, y=-0.8, label="*",fontface="bold",color = "red") +


annotate(geom = "text", x=3.65, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=3.74, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.83, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=3.92, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=4.01, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=4.10, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=4.19, y=-0.8, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=4.28, y=-0.8, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=4.37, y=-0.8, label="*",fontface="bold",color = "red") +

scale_x_discrete(expand = expansion(add = 0.6)) +
scale_fill_brewer(palette="Blues", direction=-1) +
theme_test() + labs(y = "FW [(ipsi-contra)/contra]", x = "", fill = "Location") + 
ylim(-0.8,7) -> fw_normalized_all_boxplot

# Display figure
fw_normalized_all_boxplot

# Save figure
ggsave(file.path(plot_dir, "fw_normalized_all_supratentorial_boxplot.png"), plot = fw_normalized_all_boxplot, dpi = 300, width = 6, height = 5)

### Plot average of relative free-water by time point and shell (only 3-5 days, 1 month)


In [ ]:
df_patients_supratentorial_all_tp1_2 <- df_patients_supratentorial_all[df_patients_supratentorial_all$ses_id %in% c("3-5 days", "1 month"), ]

In [ ]:
#tiff("../plots/fw_normalized_all_boxplot_tp1_2.tiff", units="in", width=6, height=5, res=300)
df_patients_supratentorial_all_tp1_2 %>% 
ggplot(aes(x=ses_id, y=fw, fill=location)) + 
annotate(geom = "rect", xmin= 0.55, xmax= 1.45, ymin=0, ymax = 0.131, color = "black", alpha=0.1) +
annotate(geom = "rect", xmin= 1.55, xmax= 2.45, ymin=0, ymax = 0.187, color = "black", alpha=0.2) +
geom_hline(yintercept=0, linetype = "solid", color = "red") +
geom_boxplot(position = position_dodge(width=0.8), width=0.6) +
annotate(geom = "text", x=0.65, y=-0.6, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=0.74, y=-0.6, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=0.83, y=-0.6, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=0.92, y=-0.6, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=1.01, y=-0.6, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=1.28, y=-0.6, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=1.37, y=-0.6, label="*",fontface="bold",color = "red") +

annotate(geom = "text", x=1.65, y=-0.6, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=1.74, y=-0.6, label="*",fontface="bold",color = "red") +

scale_x_discrete(expand = expansion(add = 0.6)) +
scale_fill_brewer(palette="Blues", direction=-1) +
theme_test() + labs(y = "FW [(ipsi-contra)/contra]", x = "", fill = "Location") + 
ylim(-0.7,2.5)
#dev.off()


### Calculate mean and SD of relative FA-t

In [ ]:
df_fat_per_timepoint <- df_patients_supratentorial_all %>% group_by(ses_id) %>% summarise(mean(fat), sd(fat))

df_fat_per_timepoint

write.csv(df_fat_per_timepoint, file.path(output_dir, "df_fat_per_timepoint_supratentorial.csv"), row.names = FALSE)

### Plot average of relative FA-t by time point

In [ ]:
#tiff("../plots/fat_normalized_all_barplot.tiff", units="in", width=6, height=5, res=300)
df_fat_per_timepoint %>% 
ggplot(aes(x=ses_id, y=`mean(fat)`, fill = ses_id)) + 
geom_bar(color="black",position = "dodge", stat="summary", fun = "mean") +
geom_errorbar(aes(ymin=`mean(fat)`-`sd(fat)`, ymax=`mean(fat)`+`sd(fat)`), width=.2, position=position_dodge(.9)) +
geom_signif(comparisons = list(c("3-5 days", "1 month")),annotation = "***") +
geom_signif(comparisons = list(c("3-5 days", "3 months")),y_position=0.085, annotation = "*") +
geom_signif(comparisons = list(c("1 month", "1 year")), y_position=0.13, annotation = "***") +
geom_signif(comparisons = list(c("3 months", "1 year")), y_position=0.11,annotation = "*") +
geom_hline(yintercept=0, linetype = "solid", color = "red") +
scale_fill_brewer(palette="Reds", direction=-1) +
theme_test() + labs(y = expression(FA[T]*" [(ipsi-contra)/contra]"), x = "", fill = "Time Point") + 
#theme(legend.position = "none") + 
ylim(-0.23,0.17) -> fat_normalized_barplot

# Display figure
fat_normalized_barplot

# Save figure
ggsave(file.path(plot_dir, "fat_normalized_all_supratentorial_barplot.png"), plot = fat_normalized_barplot, dpi = 300, width = 6, height = 5)

### Plot average of relative FA-t by time point and shell

In [ ]:
# Create figure
df_patients_supratentorial_all %>% 
ggplot(aes(x=ses_id, y=fat, fill=location)) +
annotate(geom = "rect", xmin= 0.55, xmax= 1.45, ymin=0, ymax = -0.062, color = "black", alpha=0.1) +
annotate(geom = "rect", xmin= 1.55, xmax= 2.45, ymin=0, ymax = -0.090, color = "black", alpha=0.2) +
annotate(geom = "rect", xmin= 2.55, xmax= 3.45, ymin=0, ymax = -0.077, color = "black", alpha=0.3) +
annotate(geom = "rect", xmin= 3.55, xmax= 4.45, ymin=0, ymax = -0.057, color = "black", alpha=0.4) + 
geom_hline(yintercept=0, linetype = "solid", color = "red") +
geom_boxplot(position = position_dodge(width=0.8), width=0.6) +
annotate(geom = "text", x=0.65, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=0.74, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=1.19, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=1.28, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=1.37, y=0.3, label="*",fontface="bold",color = "red") +

annotate(geom = "text", x=1.65, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=1.74, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=1.83, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=1.92, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=2.01, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=2.10, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=2.19, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=2.28, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=2.37, y=0.3, label="*",fontface="bold",color = "red") +

annotate(geom = "text", x=2.65, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=2.74, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=2.83, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=2.92, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.01, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=3.10, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.19, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.28, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.37, y=0.3, label="*",fontface="bold",color = "red") +

annotate(geom = "text", x=3.65, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=3.74, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=3.83, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=3.92, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=4.01, y=0.3, label="*",fontface="bold",color = "red") +
annotate(geom = "text", x=4.10, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=4.28, y=0.3, label="*",fontface="bold",color = "red") + 
annotate(geom = "text", x=4.37, y=0.3, label="*",fontface="bold",color = "red") + 


scale_x_discrete(expand = expansion(add = 0.6)) +
scale_fill_brewer(palette="Reds", direction=-1) +
theme_test() + labs(y = expression(FA[T]*" [(ipsi-contra)/contra]"), x = "", fill = "Location") + 
ylim(-0.6,0.47) -> fat_normalized_all_boxplot

# Display figure
fat_normalized_all_boxplot

# Save figure
ggsave(file.path(plot_dir, "fat_normalized_all_supratentorial_boxplot.png"), plot = fat_normalized_all_boxplot, dpi = 300, width = 6, height = 5)

## Plot average diffusion measures by shell for each time point separately

### Load data

In [ ]:
# Session 1
df_patients_supratentorial_ses1 <- df_esoc_norm[df_esoc_norm$group == "pat" & df_esoc_norm$ses_id == "1" & df_esoc_norm$lesion_location_category %in% c("cortical","subcortical","cortical_subcortical"), ]
df_patients_supratentorial_ses1 <- drop_na(df_patients_supratentorial_ses1, "lesionanalysis_lesion_mean_FW")
df_patients_supratentorial_ses1$location <- factor(df_patients_supratentorial_ses1$location, levels = c("lesion","2mm","4mm","6mm","8mm","10mm","12mm","14mm","16mm"))

#df$sex, levels = 0:1, labels = c("female", "male")

# Session 2
df_patients_supratentorial_ses2 <- df_esoc_norm[df_esoc_norm$group == "pat" & df_esoc_norm$ses_id == "2" & df_esoc_norm$lesion_location_category %in% c("cortical","subcortical","cortical_subcortical"), ]
df_patients_supratentorial_ses2 <- drop_na(df_patients_supratentorial_ses2, c("sub_id", "lesionanalysis_lesion_mean_FW"))
df_patients_supratentorial_ses2$location <- factor(df_patients_supratentorial_ses2$location, levels = c("lesion","2mm","4mm","6mm","8mm","10mm","12mm","14mm","16mm"))

# Session 3
df_patients_supratentorial_ses3 <- df_esoc_norm[df_esoc_norm$group == "pat" & df_esoc_norm$ses_id == "3" & df_esoc_norm$lesion_location_category %in% c("cortical","subcortical","cortical_subcortical"), ]
df_patients_supratentorial_ses3 <- drop_na(df_patients_supratentorial_ses3, c("sub_id", "lesionanalysis_lesion_mean_FW"))
df_patients_supratentorial_ses3$location <- factor(df_patients_supratentorial_ses3$location, levels = c("lesion","2mm","4mm","6mm","8mm","10mm","12mm","14mm","16mm"))

# Session 4
df_patients_supratentorial_ses4 <- df_esoc_norm[df_esoc_norm$group == "pat" & df_esoc_norm$ses_id == "4" & df_esoc_norm$lesion_location_category %in% c("cortical","subcortical","cortical_subcortical"), ]
df_patients_supratentorial_ses4 <- drop_na(df_patients_supratentorial_ses4, c("sub_id", "lesionanalysis_lesion_mean_FW"))
df_patients_supratentorial_ses4$location <- factor(df_patients_supratentorial_ses4$location, levels = c("lesion","2mm","4mm","6mm","8mm","10mm","12mm","14mm","16mm"))


### Free-water - Session 1

In [ ]:
# Create figure
df_patients_supratentorial_ses1 %>% 
ggplot(aes(x=location, y=fw, fill=location)) + 
geom_boxplot() + 
geom_jitter(width=0.1,alpha=0.2) +
geom_hline(yintercept=0, linetype = "dashed", color = "red") +
scale_fill_brewer(palette="Blues", direction=-1) +
theme_test() + labs(y = "FW [(ipsi-contra)/contra]", x = "Location", fill = "Location") + 
theme(legend.position = "none") + 
ylim(-0.7,7) -> fw_normalized_ses1_boxplot

# Display figure
fw_normalized_ses1_boxplot

# Save figure
ggsave(file.path(plot_dir, "fw_normalized_ses1_supratentorial_boxplot.png"), plot = fw_normalized_ses1_boxplot, dpi = 300, width = 6, height = 5)

### Free-water - Session 2

In [ ]:
# Create figure
df_patients_supratentorial_ses2 %>% 
ggplot(aes(x=location, y=fw, fill=location)) + 
geom_boxplot() + 
geom_jitter(width=0.1,alpha=0.2) +
geom_hline(yintercept=0, linetype = "dashed", color = "red") +
scale_fill_brewer(palette="Blues", direction=-1) +
theme_test() + labs(y = "FW [(ipsi-contra)/contra]", x = "Location", fill = "Location") + 
theme(legend.position = "none") + 
ylim(-0.7,7) -> fw_normalized_ses2_boxplot

# Display figure
fw_normalized_ses2_boxplot

# Save figure
ggsave(file.path(plot_dir, "fw_normalized_ses2_supratentorial_boxplot.png"), plot = fw_normalized_ses2_boxplot, dpi = 300, width = 6, height = 5)

### Free-water - Session 3

In [ ]:
# Create figure
df_patients_supratentorial_ses3 %>% 
ggplot(aes(x=location, y=fw, fill=location)) + 
geom_boxplot() + 
geom_jitter(width=0.1,alpha=0.2) +
geom_hline(yintercept=0, linetype = "dashed", color = "red") +
scale_fill_brewer(palette="Blues", direction=-1) +
theme_test() + labs(y = "FW [(ipsi-contra)/contra]", x = "Location", fill = "Location") + 
theme(legend.position = "none") + 
ylim(-0.7,7) -> fw_normalized_ses3_boxplot

# Display figure
fw_normalized_ses3_boxplot

# Save figure
ggsave(file.path(plot_dir, "fw_normalized_ses3_supratentorial_boxplot.png"), plot = fw_normalized_ses3_boxplot, dpi = 300, width = 6, height = 5)

### Free-water - Session 4

In [ ]:
# Create figure
df_patients_supratentorial_ses4 %>% 
ggplot(aes(x=location, y=fw, fill=location)) + 
geom_boxplot() + 
geom_jitter(width=0.1,alpha=0.2) +
geom_hline(yintercept=0, linetype = "dashed", color = "red") +
scale_fill_brewer(palette="Blues", direction=-1) +
theme_test() + labs(y = "FW [(ipsi-contra)/contra]", x = "Location", fill = "Location") + 
theme(legend.position = "none") + 
ylim(-0.5,7) -> fw_normalized_ses4_boxplot

# Display figure
fw_normalized_ses4_boxplot

# Save figure
ggsave(file.path(plot_dir, "fw_normalized_ses4_supratentorial_boxplot.png"), plot = fw_normalized_ses4_boxplot, dpi = 300, width = 6, height = 5)

### FA-t - Session 1

In [ ]:
# Create figure
df_patients_supratentorial_ses1 %>% 
ggplot(aes(x=location, y=fat, fill=location)) + 
geom_boxplot() + 
geom_jitter(width=0.1,alpha=0.2) +
geom_hline(yintercept=0, linetype = "dashed", color = "red") +
scale_fill_brewer(palette="Reds", direction=-1) +
theme_test() + labs(y = expression(FA[T]*" [(ipsi-contra)/contra]"), x = "Location", fill = "Location") + 
theme(legend.position = "none") + 
ylim(-0.6,0.5) -> fat_normalized_ses1_boxplot

# Display figure
fat_normalized_ses1_boxplot

# Save figure
ggsave(file.path(plot_dir, "fat_normalized_ses1_supratentorial_boxplot.png"), plot = fat_normalized_ses1_boxplot, dpi = 300, width = 6, height = 5)

### FA-t - Session 2

In [ ]:
# Create figure
df_patients_supratentorial_ses2 %>% 
ggplot(aes(x=location, y=fat, fill=location)) + 
geom_boxplot() + 
geom_jitter(width=0.1,alpha=0.2) +
geom_hline(yintercept=0, linetype = "dashed", color = "red") +
scale_fill_brewer(palette="Reds", direction=-1) +
theme_test() + labs(y = expression(FA[T]*" [(ipsi-contra)/contra]"), x = "Location", fill = "Location") + 
theme(legend.position = "none") + 
ylim(-0.6,0.5) -> fat_normalized_ses2_boxplot

# Display figure
fat_normalized_ses2_boxplot

# Save figure
ggsave(file.path(plot_dir, "fat_normalized_ses2_supratentorial_boxplot.png"), plot = fat_normalized_ses2_boxplot, dpi = 300, width = 6, height = 5)

### FA-t - Session 3

In [ ]:
# Create figure
df_patients_supratentorial_ses3 %>% 
ggplot(aes(x=location, y=fat, fill=location)) + 
geom_boxplot(show.legend = FALSE) + 
geom_jitter(width=0.1,alpha=0.2) +
geom_hline(yintercept=0, linetype = "dashed", color = "red") +
scale_fill_brewer(palette="Reds", direction=-1) +
theme_test() + labs(y = expression(FA[T]*" [(ipsi-contra)/contra]"), x = "Location", fill = "Location") + 
theme(legend.position = "none") + 
ylim(-0.6,0.5) -> fat_normalized_ses3_boxplot

# Display figure
fat_normalized_ses3_boxplot

# Save figure
ggsave(file.path(plot_dir, "fat_normalized_ses3_supratentorial_boxplot.png"), plot = fat_normalized_ses3_boxplot, dpi = 300, width = 6, height = 5)

### FA-t - Session 4

In [ ]:
# Create figure
df_patients_supratentorial_ses4 %>% 
ggplot(aes(x=location, y=fat, fill=location)) + 
geom_boxplot(show.legend = FALSE) + 
geom_jitter(width=0.1,alpha=0.2) +
geom_hline(yintercept=0, linetype = "dashed", color = "red") +
scale_fill_brewer(palette="Reds", direction=-1) +
theme_test() + labs(y = expression(FA[T]*" [(ipsi-contra)/contra]"), x = "Location", fill = "Location") + 
theme(legend.position = "none") +
ylim(-0.6,0.5) -> fat_normalized_ses4_boxplot

# Display figure
fat_normalized_ses4_boxplot

# Save figure
ggsave(file.path(plot_dir, "fat_normalized_ses4_supratentorial_boxplot.png"), plot = fat_normalized_ses4_boxplot, dpi = 300, width = 6, height = 5)